# API (No Key) Basics

## 1. Learning Goals
By the end of this section, you will be able to:
- Call a public API without authentication.
- Pass query parameters in a request.
- Handle pagination to collect multiple pages of results.
- Convert JSON responses into a table (e.g., DataFrame).

---

## 2. Request → Response Mental Model
Think of an API call as a **conversation**:

- **Endpoint**: the exact URL you call.  
- **Method**: often `GET` for retrieving data.  
- **Query parameters**: key–value pairs after `?` in the URL that filter or shape results.  
- **Status code**: tells you if the request worked (`200 OK`) or failed (`4xx/5xx`).  
- **Body**: the data returned, usually in JSON format.  

Example request URL:

This tutorial uses the [Caltrans API](https://caltrans-gis.dot.ca.gov/arcgis/rest/services)
---

## 3. Vocabulary for APIs
| Term | Why it matters |
|------|----------------|
| **Endpoint** | The specific URL where you send a request. |
| **Query parameters** | Extra information in the request URL to filter, limit, or sort results. |
| **HTTP status code** | A fast way to see if your call succeeded (`200`), failed (`404` not found), or had an error (`500` server). |
| **Pagination** | Splitting results across multiple pages; requires parameters like `offset` and `limit`. |
| **Rate limit** | Restriction on how many calls you can make per minute/hour. |
| **Schema** | The structure of the data (field names, data types) that the API returns. |

---


- APIs use a dedicated URL that returns data that is often in a JSON format. 

- We can parse the json file and to get our data. Read more about JSON [HERE]('https://realpython.com/python-json/').

In [ ]:
# Add libraries
import requests
import pprint as pp
import pandas as pd

### API example - no key

In this example we scrape cctv footage from CA highways and put the data into a dataframe. 

No API key required for caltrans-gis.dot.ca.gov

In [5]:
# function to grab data from a resource url
def getdata(url):
    try:
        response=requests.get(url)
        return response.json()
    except:
        return response.status_code

### Using the above function, get the cctv data.

In [6]:
# get data from ca.gov api explorer
cctv=getdata('https://caltrans-gis.dot.ca.gov/arcgis/rest/services/CHhighway/CCTV/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

## JSON Data Format

Most APIs return data in **JSON** (JavaScript Object Notation).  
It often looks like a **dictionary** with keys and values.

Example:

```json
{
  "fields": [...],
  "features": [
    { "attributes": {...}, "geometry": {...} },
    { "attributes": {...}, "geometry": {...} }
  ]
}


In [13]:
type(cctv)

dict

In [8]:
# view the data
cctv.keys()

dict_keys(['objectIdFieldName', 'globalIdFieldName', 'geometryType', 'spatialReference', 'fields', 'features', 'exceededTransferLimit'])

In [9]:
# view first item in features key
cctv['features'][0]

{'attributes': {'OBJECTID': 1,
  'index_': 1,
  'recordDate': 1626912000000,
  'recordTime': '06:50:03',
  'recordEpoch': 1626961803,
  'district': 1,
  'locationName': 'SR-20 : At SR-1 - Looking East (C020)',
  'nearbyPlace': 'Fort Bragg',
  'longitude': -123.80779,
  'latitude': 39.42002,
  'elevation': 95.0,
  'direction': 'East',
  'county': 'Mendocino',
  'route': 'SR-20',
  'routeSuffix': None,
  'postmilePrefix': 'R',
  'postmile': 0.01,
  'Postmile_Suffix': None,
  'Odometer': 0.01,
  'inService': 'True',
  'imageDescription': None,
  'streamingVideoURL': None,
  'currentImageUpdateFrequency': '15',
  'currentImageURL': 'https://cwwp2.dot.ca.gov/data/d1/cctv/image/sr20atsr1lookingeast/sr20atsr1lookingeast.jpg'},
 'geometry': {'x': -123.80778999963576, 'y': 39.42002000007047}}

Search the cctv json data which will be in a dictionary format. We will need to get the 'features' data.

In [10]:
# make a list of dictionaries that include cctv 'features'
features=[dict['attributes'] for dict in cctv['features']]

In [11]:
# create a dataframe from the lsit of dictionaries
cctvdf=pd.DataFrame.from_records(features)

In [12]:
# view dataframe
cctvdf.head()

,OBJECTID,index_,recordDate,recordTime,recordEpoch,district,locationName,nearbyPlace,longitude,latitude,...,routeSuffix,postmilePrefix,postmile,Postmile_Suffix,Odometer,inService,imageDescription,streamingVideoURL,currentImageUpdateFrequency,currentImageURL
0,1,1,1626912000000,06:50:03,1626961803,1,SR-20 : At SR-1 - Looking East (C020),Fort Bragg,-123.80779,39.42002,...,None,R,0.01,None,0.01,True,None,None,15,https://cwwp2.dot.ca.gov/data/d1/cctv/image/sr...
1,2,9,1715040000000,09:43:18,1715100198,1,SR-20 : West Of US-101 - Looking East (C007),Willits,-123.36904,39.40581,...,None,None,32.21,None,32.16,True,None,None,10,https://cwwp2.dot.ca.gov/data/d1/cctv/image/sr...
2,3,11,1715040000000,09:43:32,1715100212,1,SR-20 : West Of US-101 - Looking West (C007),Willits,-123.36904,39.40581,...,None,None,32.21,None,32.16,True,None,None,10,https://cwwp2.dot.ca.gov/data/d1/cctv/image/sr...
3,4,14,1728864000000,11:45:55,1728931555,1,SR-20 : East Of US-101 - Looking East (C006),Ukiah,-123.16703,39.23780,...,None,None,35.68,None,35.57,True,None,None,15,https://cwwp2.dot.ca.gov/data/d1/cctv/image/sr...
4,5,16,1728864000000,11:46:11,1728931571,1,SR-20 : East Of US-101 - Looking West (C006),Ukiah,-123.16703,39.23780,...,None,None,35.68,None,35.57,True,None,None,15,https://cwwp2.dot.ca.gov/data/d1/cctv/image/sr...
